<a href="https://colab.research.google.com/github/ajaykumar080286/DeepLearning/blob/master/age_gender_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [36]:
import zipfile
zip = zipfile.ZipFile("/content/UTKFace.zip",'r')
zip.extractall("/content")
zip.close()

In [37]:
folder_path = '/content/UTKFace'

In [38]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [39]:
img_path

['64_0_0_20170117194627251.jpg.chip.jpg',
 '64_0_0_20170120222952994.jpg.chip.jpg',
 '44_1_0_20170117190014098.jpg.chip.jpg',
 '64_0_0_20170120222811411.jpg.chip.jpg',
 '64_0_0_20170120223401173.jpg.chip.jpg',
 '44_1_0_20170117190015938.jpg.chip.jpg']

In [40]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [41]:
df

,age,gender,img
0,64,0,64_0_0_20170117194627251.jpg.chip.jpg
1,64,0,64_0_0_20170120222952994.jpg.chip.jpg
2,44,1,44_1_0_20170117190014098.jpg.chip.jpg
3,64,0,64_0_0_20170120222811411.jpg.chip.jpg
4,64,0,64_0_0_20170120223401173.jpg.chip.jpg
5,44,1,44_1_0_20170117190015938.jpg.chip.jpg


In [42]:
train_df = df.iloc[:4]
test_df = df.iloc[4:]

In [43]:
test_df

,age,gender,img
4,64,0,64_0_0_20170120223401173.jpg.chip.jpg
5,44,1,44_1_0_20170117190015938.jpg.chip.jpg


In [44]:
train_df

,age,gender,img
0,64,0,64_0_0_20170117194627251.jpg.chip.jpg
1,64,0,64_0_0_20170120222952994.jpg.chip.jpg
2,44,1,44_1_0_20170117190014098.jpg.chip.jpg
3,64,0,64_0_0_20170120222811411.jpg.chip.jpg


In [45]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [46]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 4 validated image filenames.
Found 2 validated image filenames.


In [47]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [48]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [49]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [50]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
def wrap_multi_output_generator(generator):
    for x, y_list in generator:
        yield x, tuple(y_list)

wrapped_train_generator = wrap_multi_output_generator(train_generator)
wrapped_test_generator = wrap_multi_output_generator(test_generator)

model.fit(wrapped_train_generator,batch_size=1, epochs=1, validation_data=wrapped_test_generator)

     63/Unknown 135s 2s/step - age_loss: 9.7343 - age_mae: 9.7343 - gender_accuracy: 0.6950 - gender_loss: 1.4774 - loss: 155.9992